Nettoyage d’un texte 

In [ ]:
import pandas as pd
url = "https://github.com/GU4243-ADS/spring2018-project1-ginnyqg/raw/master/data/spooky.csv"
# Load the dataset
file = 'spooky.csv'
df = pd.read_csv(url)

# Display the first 10 samples
print(df.head(10))

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
import re
from nltk.corpus import wordnet


repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
repl = r'\1\2\3'

def replace(word):
    if wordnet.synsets(word):
        return word
    repl_word = repeat_regexp.sub(repl, word)

    if repl_word != word:
        return replace(repl_word)
    else:
        return repl_word
    
def handle_repetitive_chars(text):
    words = text.split()
    new_words = []
    for word in words:
        new_words.append(replace(word))
    return ' '.join(new_words)

df['text'] = df['text'].apply(handle_repetitive_chars)

print(handle_repetitive_chars("cooooooool"))

print(df.head(10))

In [ ]:
homoglyphs_map = {
    '$': 's', '0': 'o', '1': 'l', '3': 'e', '@': 'a', '5': 's', '7': 't', 
    '8': 'b', '|': 'i', '¢': 'c', '¡': 'i', 'µ': 'u', 'ß': 'b', '€': 'e',
    '!':'i','£':'l',
        
    'Α': 'A', 
    'Β': 'B', 
    'Ε': 'E', 
    'Ζ': 'Z', 
    'Η': 'H', 
    'Ι': 'I', 
    'Κ': 'K', 
    'Μ': 'M', 
    'Ν': 'N', 
    'Ο': 'O', 
    'Ρ': 'P', 
    'Τ': 'T', 
    'Χ': 'X', 

    'А': 'A',
    'В': 'B',
    'С': 'C',
    'Е': 'E',
    'Н': 'H',
    'К': 'K',
    'М': 'M',
    'О': 'O',
    'Р': 'P',
    'Т': 'T',
    'Х': 'X'
}

def replace_homoglyphs(text):
    return ''.join(homoglyphs_map.get(char, char) for char in text)


print(replace_homoglyphs("$tupid"))

df['text'] = df['text'].apply(replace_homoglyphs)

In [37]:
from bs4 import BeautifulSoup

def clean_special_entries(text):
    text = re.sub(r'http\S+|www\S+', ' <URL> ', text)  
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', ' <EMAIL> ', text)  
    text = BeautifulSoup(text, "html.parser").get_text()  
    return text

df['text'] = df['text'].apply(clean_special_entries)


In [38]:
df['text'] = df['text'].str.lower()

import string

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

df['text'] = df['text'].apply(remove_punctuation)


In [ ]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in stop_words])

df['text'] = df['text'].apply(remove_stopwords)

In [41]:
df=df.head(100)

In [42]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from deep_translator import GoogleTranslator
from collections import Counter
import pandas as pd


# Example DataFrame (replace with your actual data)
# df = pd.read_csv('your_data.csv')

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

# Detect language for each text entry
df['language'] = df['text'].apply(detect_language)

# Determine the most frequent language in your dataset
most_frequent_lang = Counter(df['language']).most_common(1)[0][0]

# Cache translators to avoid creating a new instance each time
translators = {}

def translate_text(text, src_lang):
    # Only translate if the source language is not the majority and not unknown
    if src_lang != most_frequent_lang and src_lang != "unknown":
        # Use a cached translator if available; otherwise, create and cache one
        if src_lang not in translators:
            translators[src_lang] = GoogleTranslator(source=src_lang, target=most_frequent_lang)
        return translators[src_lang].translate(text)
    return text

# Apply translation row by row
df['text'] = df.apply(lambda row: translate_text(row['text'], row['language']), axis=1)


In [43]:
df['text'] = df['text'].apply(lambda text: re.sub(r'[^a-zA-Z0-9\s]', '', text))


In [44]:
def remove_repeated_words(text):
    words = text.split()
    return " ".join(sorted(set(words), key=words.index))  # Keeps first occurrence of each word

df['text'] = df['text'].apply(remove_repeated_words)
df['text'] = df['text'].str.strip()  # Remove leading/trailing spaces


In [ ]:
print(df.head(10))

In [ ]:
df.to_csv("cleaned_spooky.csv", index=False)
print("Preprocessing complete. Data saved as cleaned_spooky.csv")


Segmentation

In [47]:
import spacy

In [ ]:
nltk.download('punkt')

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import spacy

# Download the necessary NLTK tokenizer data (if not already downloaded)
nltk.download('punkt')

# Load SpaCy English model (only needed if you plan to use SpaCy later)
nlp = spacy.load("en_core_web_sm")

# Read the CSV file
file = 'cleaned_spooky.csv'
df = pd.read_csv(file)

print("First 10 rows before cleaning:")
print(df.head(10))

# Drop rows where the 'text' column is NaN
df = df.dropna(subset=['text'])

# Option A: Using NLTK's word_tokenize for tokenization
df['tokens'] = df['text'].apply(word_tokenize)

# Option B (Alternative): Using SpaCy's tokenizer with nlp.pipe (better for large datasets)
# Uncomment the following block and comment out Option A if you prefer SpaCy
# tokens = []
# for doc in nlp.pipe(df['text'], batch_size=50, n_process=-1):
#     tokens.append([token.text for token in doc])
# df['tokens'] = tokens

print("First 10 rows after tokenization:")
print(df.head(10))


In [52]:
import spacy

nlp = spacy.load("en_core_web_sm")

def rule_based_tokenization(text):
    doc = nlp(text)
    return [token.text for token in doc]

df['rule_based_tokens'] = df['text'].apply(rule_based_tokenization)

In [ ]:
print(df.head(10))

In [ ]:

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def subword_tokenization(text):
    return tokenizer.tokenize(text)

df['subword_tokens'] = df['text'].apply(subword_tokenization)

print(df.head(10))


 Reconnaissance d'entité nommée 

In [55]:
def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

df['entities'] = df['text'].apply(extract_entities)

In [ ]:
def pos_tagging(text):
    doc = nlp(text)
    return [(token.text, token.pos_) for token in doc]

df['pos_tags'] = df['text'].apply(pos_tagging)
print(df.head(10))

E. Réduction des formes

In [57]:
from nltk.stem import WordNetLemmatizer, PorterStemmer

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

def stem_text(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

df['lemmatized_text'] = df['text'].apply(lemmatize_text)
df['stemmed_text'] = df['text'].apply(stem_text)

 Analyse des fréquences 

In [ ]:
df['contains_great'] = df['text'].apply(lambda x: 'great' in x.lower())
great_counts = df.groupby('author')['contains_great'].sum()
print(great_counts)

In [ ]:
import matplotlib.pyplot as plt
from pywaffle import Waffle

data = great_counts.to_dict()
fig = plt.figure(
    FigureClass=Waffle,
    rows=5,
    columns=10,
    values=data,
    legend={'loc': 'upper left', 'bbox_to_anchor': (1, 1)},
    title={'label': 'Occurrences of "great" by Author', 'loc': 'left'}
)
plt.show()

In [ ]:
df['contains_impossible'] = df['text'].apply(lambda x: 'impossible' in x.lower())
impossible_counts = df.groupby('author')['contains_impossible'].sum()
print(impossible_counts)

In [ ]:
data = impossible_counts.to_dict()

if sum(data.values()) == 0:
    print("No data available to plot the waffle chart.")
else:
    fig = plt.figure(
        FigureClass=Waffle,
        rows=5,
        columns=10,
        values=data,
        legend={'loc': 'upper left', 'bbox_to_anchor': (1, 1)},
        title={'label': 'Occurrences of "impossible" by Author', 'loc': 'left'}
    )
    plt.show()


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

for author in df['author'].unique():
    text = ' '.join(df[df['author'] == author]['text'])
    wordcloud = WordCloud(width=800, height=400).generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Word Cloud for {author}')
    plt.axis('off')
    plt.show()

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

def get_sentiment_words(text):
    words = text.split()
    positive_words = [word for word in words if sia.polarity_scores(word)['compound'] > 0]
    negative_words = [word for word in words if sia.polarity_scores(word)['compound'] < 0]
    return positive_words, negative_words

df['positive_words'], df['negative_words'] = zip(*df['text'].apply(get_sentiment_words))

# Get top 100 positive and negative words
all_positive_words = [word for sublist in df['positive_words'] for word in sublist]
all_negative_words = [word for sublist in df['negative_words'] for word in sublist]

top_positive_words = pd.Series(all_positive_words).value_counts().head(100)
top_negative_words = pd.Series(all_negative_words).value_counts().head(100)
# Generate WordCloud for positive words


# Generate WordCloud for top positive words
positive_wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='Greens').generate_from_frequencies(top_positive_words.to_dict())
plt.figure(figsize=(10, 5))
plt.imshow(positive_wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Top 100 Positive Words")
plt.show()

# Generate WordCloud for top negative words
negative_wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='Reds').generate_from_frequencies(top_negative_words.to_dict())
plt.figure(figsize=(10, 5))
plt.imshow(negative_wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Top 100 Negative Words")
plt.show()